# Hansd-on Machine Learning with Scikit-Learn & TensorFlow

기본설정

In [1]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf ##배워보자 !!!

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

# 10장 인공 신경망 소개

## 10.0 Introduction

- 많은 발명품이 자연으로부터 영감을 받듯이 지능적인 기계에 대한 영감을 얻기 위해서는 죄의 구조를 살펴봐야 한다.  
- 이것이 인공 신경망(ANN)을 만들어낸 핵심 아이디어다.  
- ANN = artificial neural networks  
- 이미지 분류, 음성인식, 추천, 게임 등 아주 복잡한 대규모 머신러닝 문제를 다루는 데 적합하다.  



- 이 장에서는 인공 신경망의 초창기 구조를 간단히 소개하고, 다층 퍼셉트론(Multi-Layer Perceptron,MLP)을 설명하고 MNIST 숫자 분류 문제를 텐서플로를 사용해 구현한다.


## 10.1 생물학적 뉴런에서 인공 뉴런까지

- 인공 신경망은 규모가 크고 복잡한 문제에서 다른 머신러닝 기법보다 좋은 성능을 낸다.  

### 10.1.1 생물학적 뉴런

### 10.1.2 뉴런을 사용한논리 연산

그림 10-3

1. 간단한 항등 함수
2. 논리곱 연산
3. 논리합 연산
4. 어떤 입력이 뉴런의 활성화를 억제할 수 있다고 가정하면 이런 연산도 가능함

### 10.1.3 퍼셉트론

- 퍼셉트론은 TLU(threshold logic unit)라는 인공 뉴런을 기반으로 한다.

- 입력과 출력이 어떤 숫자고 각각의 입력 연결은 가중치와 연관되어 있다.
- TLU는 입력의 가중치 합을 계산하고, 그런 다음 계산된 합에 계단 함수를 적용하여 그 결과를 출력한다.

$$z = w_1x_1 + w_2x_2 + \cdots + w_nx_n = W^T\cdot x$$
$$h_W(x) = step(z) = step(W^T\cdot x)$$

그림 10-4

퍼셉트론에서 일반적으로 사용하는 계단 함수

$$heaviside(z) = \begin{cases} 0, & \mbox{if }z<0  \\
1, & \mbox{if } z\geq 0\end{cases}$$  


$$sgn(z)=\begin{cases} 
-1, & \mbox{if } z<0  \\
0, & \mbox{if } z= 0 \\
1, & \mbox{if } z\geq 0
\end{cases}$$

- 하나의 TLU는 간단한 선형 이진 분류 문제에 사용할 수 있다. 입력의 선형 조합을 계산해서 그 결과가 임곗값을 넘어서면 양성 클래스를 출력하고 그렇지 않으면 음성 클래스를 출력한다. TLU를 훈련시킨다는 것은 최적의 $w_0$$w_1$$w_2$를 찾는다는 뜻이다.(특성이 2개라면)

- 퍼셉트론은 층이 하나뿐인 TLU로 구성된다.
- 각 뉴런은 모든 입력에 연결되어 있다. 이 연결은 입력 뉴런 이라 부르는 특별한 통과 뉴런을 사용해 표현되곤 한다. 이 뉴런은 무엇이 주입되든 입력을 그냥 출력으로 통과시킨다. 보통 여기에 편향 특성이 더해진다. 전통적으로 이 편향 특성은 항상 1을 출력하는 특별한 종류의 뉴런인 편향 뉴런으로 표현된다.

퍼셉트론 학습 규칙(가중치 업데이트)

$$W_{i,j}^{(next step)} = W_{i,j} + \eta(y_i - \hat{y_i})x_i$$

- $W_{i,j}$는 $i$번째 입력 뉴런과 $j$번째 출력 뉴런 사이를 연결하는 가중치입니다.
- $x_i$는 현재 훈련 샘플의 $i$번째 뉴런의 입력값입니다.
- $\hat{y_i}$는 현재 훈련 샘플의 $j$번째 출력 뉴련의 출력값입니다.
- $y_i$는 현재 훈련 샘플의 $j$번째 출력 뉴런의 타깃값입니다.
- $\eta$는 학습률입니다.

__퍼셉트론 수렴 이론__  
훈련 샘플이 선형적으로 구분될 수 있다면 이 알고리즘은 정답에 수렴한다. 

In [2]:
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2,3)]
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])

c:\users\student\documents\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


퍼셉트론 학습 알고리즘은 확률적 경사 하강법과 비슷하다.  
SGDClassifier로 loss="perceptron", learning_rate="constant", eta0=1, penalty=None 설정한 것이다.

로지스틱 회귀 분류기와 달리 퍼셉트론은 클래스 확률을 제공하지 않으며 고정된 임곗값을 기준으로 예측을 만든다.  
이런 이유로 퍼셉트론보다 로지스틱 회귀가 더 선호된다. 

하지만 배타적 논리합(XOR) 분류 문제를 해결 할 수 없다.

이를 해결하는 방법은 여러 퍼셉트론을 쌓아올려 일부 제약을 줄일 수 있고, 이를 다층 퍼셉트론(Multi-Layer Perceptron, MLP)라 한다.

### 10.1.4 다층 퍼셉트론과 역전파

- 다층 퍼셉트론은 입력층 하나와 은닉층이라 불리는 하나 이상의 TLU 층과 마지막 층인 출력층으로 구성된다.
- 출력층을 제외하고 모든 층은 편향 뉴런을 포함하며 다음 층과 완전히 연결되어 있다.
- 인공 신경망의 은닉층이 2개 이상일 때 이를 심층 신경망(deep neural network, DNN)이라고 한다.

역전파 훈련 알고리즘, 혹은 후진 모드 자동 미분을 사용하는 경사 하강법으로 불리는 알고리즘으로 다층 퍼셉트론을 훈련시킨다.

훈련 샘플에 대해 역전파 알고리즘이 먼저 예측을 만들고(정방향 계산) 오차를 측정하고, 그런 다음 역방향으로 각 층을 거치면서 각 연결이 오차에 기여한 정도를 측정한다(역방향 계산). 마지막으로 이 오차가 감소하도록 가중치를 조금씩 조정한다(경사 하강법 스텝).

다양한 활성화 함수가 있다.
0. __계산 함수__


1. __로지스틱 함수__
$$\sigma(z)=1/(1+exp(-z)$$
모든 곳에 미분이 정의되어 있음


2. __하이퍼볼릭 탄젠트 함수(쌍곡 탄젠트 함수)__
$$tanh(z) = 2\sigma(2z)-1$$
로지스틱 함수처럼 이 함수도 S자 모양이고 연속적이며 미분 가능하다. 하지만 출력 범위가 -1에서 1사이이다. 그래서 훈련 초기에 각 층의 출력이 다소 정규화되는 경향이 있다(원점 주위로 몰리게 된다). 이는 종종 빠르게 수렴되도록 도와준다.


3. __ReLU 함수__
$$ReLU(z)=max(0,z)$$
이 함수는 연속적이지만 $z=0$에서 미분 가능하지 않다. 그러나 실제로는 잘 작동하고 계산 속도가 빠르다는 장점이 있다. 무엇보다 중요한 점은 출력에 최댓값이 없다는 점이 경사 하강법에 있는 일부 문제를 완와해준다.

- 다층 퍼셉트론은 각 출력이 서로 다른 이진 클래스에 대응되는 분류 문제에 자주 사용된다.  
- 클래스가 배타적일 때는 전형적으로 출력층의 활성화 함수를 소프트맥스 함수로 바꿔준다.(추정 확률을 리턴함)  
- 신호가 한 방향으로만(입력에서 출력으로) 흐르기 때문에 이런 구조를 피드포워드 신경망(feed forward neural network, FNN)이라고 한다.

## 10.2 텐서플로의 고수준 API로 다층 퍼섭트론 훈련하기

In [3]:
reset_graph()

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print(type(X_train))
print(X_train.shape)
print(X_test.shape)

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
## 0~255값이 있는데 이를 0~1의 값으로 스케일링


X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

<class 'numpy.ndarray'>
(60000, 28, 28)
(10000, 28, 28)


In [4]:
feature_cols = [tf.feature_column.numeric_column("X", shape=[28*28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                        feature_columns=feature_cols)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=40, batch_size=50, shuffle=True)
dnn_clf.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\student\\AppData\\Local\\Temp\\tmpbbde5o7b', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000385A780>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\student\AppData\Local\Temp\tmpbbde5o7b\model.ckpt.
INFO:tensorflow:loss = 122.54302, step = 1
INFO:tensorflow:global_step/sec: 339.674
INFO:tensorflow:loss = 22.393211, step = 101 (0.294 sec)
INFO:tensorflow:global_step/sec: 430.292
INFO:tensorflow:loss = 12.364537, step = 201 (0.

INFO:tensorflow:global_step/sec: 430.293
INFO:tensorflow:loss = 0.12777722, step = 7501 (0.232 sec)
INFO:tensorflow:global_step/sec: 428.449
INFO:tensorflow:loss = 2.2081718, step = 7601 (0.233 sec)
INFO:tensorflow:global_step/sec: 426.621
INFO:tensorflow:loss = 0.31195664, step = 7701 (0.234 sec)
INFO:tensorflow:global_step/sec: 432.152
INFO:tensorflow:loss = 0.43396264, step = 7801 (0.231 sec)
INFO:tensorflow:global_step/sec: 406.504
INFO:tensorflow:loss = 0.7513974, step = 7901 (0.246 sec)
INFO:tensorflow:global_step/sec: 404.859
INFO:tensorflow:loss = 0.37146246, step = 8001 (0.248 sec)
INFO:tensorflow:global_step/sec: 412.882
INFO:tensorflow:loss = 0.22147717, step = 8101 (0.241 sec)
INFO:tensorflow:global_step/sec: 406.835
INFO:tensorflow:loss = 1.6454022, step = 8201 (0.248 sec)
INFO:tensorflow:global_step/sec: 380.228
INFO:tensorflow:loss = 0.16116127, step = 8301 (0.263 sec)
INFO:tensorflow:global_step/sec: 400.962
INFO:tensorflow:loss = 0.09267668, step = 8401 (0.247 sec)
INF

INFO:tensorflow:loss = 0.033203915, step = 15601 (0.234 sec)
INFO:tensorflow:global_step/sec: 428.449
INFO:tensorflow:loss = 0.10237986, step = 15701 (0.233 sec)
INFO:tensorflow:global_step/sec: 430.292
INFO:tensorflow:loss = 0.012636917, step = 15801 (0.232 sec)
INFO:tensorflow:global_step/sec: 426.622
INFO:tensorflow:loss = 0.044916302, step = 15901 (0.234 sec)
INFO:tensorflow:global_step/sec: 428.449
INFO:tensorflow:loss = 0.0048502153, step = 16001 (0.233 sec)
INFO:tensorflow:global_step/sec: 426.621
INFO:tensorflow:loss = 0.015980924, step = 16101 (0.234 sec)
INFO:tensorflow:global_step/sec: 420.168
INFO:tensorflow:loss = 0.91442025, step = 16201 (0.238 sec)
INFO:tensorflow:global_step/sec: 449.64
INFO:tensorflow:loss = 0.2012431, step = 16301 (0.222 sec)
INFO:tensorflow:global_step/sec: 420.168
INFO:tensorflow:loss = 0.08192321, step = 16401 (0.238 sec)
INFO:tensorflow:global_step/sec: 423.012
INFO:tensorflow:loss = 0.02867841, step = 16501 (0.236 sec)
INFO:tensorflow:global_step

INFO:tensorflow:loss = 0.051715836, step = 23701 (0.248 sec)
INFO:tensorflow:global_step/sec: 381.68
INFO:tensorflow:loss = 0.053707756, step = 23801 (0.263 sec)
INFO:tensorflow:global_step/sec: 394.633
INFO:tensorflow:loss = 0.05276909, step = 23901 (0.252 sec)
INFO:tensorflow:global_step/sec: 404.858
INFO:tensorflow:loss = 0.007084897, step = 24001 (0.247 sec)
INFO:tensorflow:global_step/sec: 406.504
INFO:tensorflow:loss = 0.009384312, step = 24101 (0.246 sec)
INFO:tensorflow:global_step/sec: 409.836
INFO:tensorflow:loss = 0.044265117, step = 24201 (0.245 sec)
INFO:tensorflow:global_step/sec: 403.226
INFO:tensorflow:loss = 0.06549963, step = 24301 (0.248 sec)
INFO:tensorflow:global_step/sec: 408.163
INFO:tensorflow:loss = 0.023700599, step = 24401 (0.244 sec)
INFO:tensorflow:global_step/sec: 429.553
INFO:tensorflow:loss = 0.057038743, step = 24501 (0.232 sec)
INFO:tensorflow:global_step/sec: 421.585
INFO:tensorflow:loss = 0.013073122, step = 24601 (0.239 sec)
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 378.788
INFO:tensorflow:loss = 0.02697596, step = 31801 (0.264 sec)
INFO:tensorflow:global_step/sec: 367.647
INFO:tensorflow:loss = 0.014603955, step = 31901 (0.272 sec)
INFO:tensorflow:global_step/sec: 440.529
INFO:tensorflow:loss = 0.013223002, step = 32001 (0.226 sec)
INFO:tensorflow:global_step/sec: 350.631
INFO:tensorflow:loss = 0.044856984, step = 32101 (0.286 sec)
INFO:tensorflow:global_step/sec: 432.526
INFO:tensorflow:loss = 0.0010049241, step = 32201 (0.230 sec)
INFO:tensorflow:global_step/sec: 430.293
INFO:tensorflow:loss = 0.0069726743, step = 32301 (0.232 sec)
INFO:tensorflow:global_step/sec: 400
INFO:tensorflow:loss = 0.0019998953, step = 32401 (0.250 sec)
INFO:tensorflow:global_step/sec: 434.783
INFO:tensorflow:loss = 0.027647933, step = 32501 (0.230 sec)
INFO:tensorflow:global_step/sec: 411.523
INFO:tensorflow:loss = 0.009040857, step = 32601 (0.243 sec)
INFO:tensorflow:global_step/sec: 449.64
INFO:tensorflow:loss = 0.0075975046, step = 

INFO:tensorflow:global_step/sec: 430.293
INFO:tensorflow:loss = 0.012334694, step = 39901 (0.232 sec)
INFO:tensorflow:global_step/sec: 432.152
INFO:tensorflow:loss = 0.009002305, step = 40001 (0.231 sec)
INFO:tensorflow:global_step/sec: 421.941
INFO:tensorflow:loss = 0.00055223325, step = 40101 (0.237 sec)
INFO:tensorflow:global_step/sec: 435.919
INFO:tensorflow:loss = 0.01923876, step = 40201 (0.229 sec)
INFO:tensorflow:global_step/sec: 420.168
INFO:tensorflow:loss = 0.003165529, step = 40301 (0.238 sec)
INFO:tensorflow:global_step/sec: 435.92
INFO:tensorflow:loss = 0.03513373, step = 40401 (0.229 sec)
INFO:tensorflow:global_step/sec: 407.498
INFO:tensorflow:loss = 0.04377911, step = 40501 (0.245 sec)
INFO:tensorflow:global_step/sec: 431.035
INFO:tensorflow:loss = 0.0083409725, step = 40601 (0.232 sec)
INFO:tensorflow:global_step/sec: 418.41
INFO:tensorflow:loss = 0.015032208, step = 40701 (0.239 sec)
INFO:tensorflow:global_step/sec: 449.64
INFO:tensorflow:loss = 0.01457405, step = 40

In [5]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Starting evaluation at 2019-01-07-00:36:57
INFO:tensorflow:Restoring parameters from C:\Users\student\AppData\Local\Temp\tmpbbde5o7b\model.ckpt-44000
INFO:tensorflow:Finished evaluation at 2019-01-07-00:36:57
INFO:tensorflow:Saving dict for global step 44000: accuracy = 0.9828, average_loss = 0.10030776, global_step = 44000, loss = 12.6971855


In [6]:
print(type(eval_results))
eval_results

<class 'dict'>


{'accuracy': 0.9828,
 'average_loss': 0.10030776,
 'loss': 12.6971855,
 'global_step': 44000}

In [7]:
y_pred_iter = dnn_clf.predict(input_fn=test_input_fn)
y_pred = list(y_pred_iter)
y_pred[0]

INFO:tensorflow:Restoring parameters from C:\Users\student\AppData\Local\Temp\tmpbbde5o7b\model.ckpt-44000


{'logits': array([-13.990679 ,  -3.6785386,  -2.487329 ,   6.299542 ,  -9.07686  ,
         -7.026543 , -16.983297 ,  24.333138 ,  -5.2661104,   3.3461878],
       dtype=float32),
 'probabilities': array([2.2708000e-17, 6.8341318e-13, 2.2491540e-12, 1.4726813e-08,
        3.0918833e-15, 2.4025031e-14, 1.1389365e-18, 1.0000000e+00,
        1.3970449e-13, 7.6821616e-10], dtype=float32),
 'class_ids': array([7], dtype=int64),
 'classes': array([b'7'], dtype=object)}

책 설명

In [8]:
config = tf.contrib.learn.RunConfig(tf_random_seed=42) # 책에는 없음

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
tf.logging.set_verbosity(tf.logging.INFO)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000E950DA0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\student\\AppData\\Local\\Temp\\tmp8cq4fsj7'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\student\AppData\Local\Temp\tmp8cq4fsj7\model.ckpt.
INFO:tensorflow:loss = 2.4005778, step = 1
INFO:tensorflow:global_step/sec: 389.105
INFO:tensorflow:loss = 0.31267592, step = 101 (0.257 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:global_step/sec: 459.137
INFO:tensorflow:loss = 0.009239808, step = 7301 (0.218 sec)
INFO:tensorflow:global_step/sec: 461.255
INFO:tensorflow:loss = 0.017138585, step = 7401 (0.217 sec)
INFO:tensorflow:global_step/sec: 459.137
INFO:tensorflow:loss = 0.010926548, step = 7501 (0.218 sec)
INFO:tensorflow:global_step/sec: 463.392
INFO:tensorflow:loss = 0.014204067, step = 7601 (0.216 sec)
INFO:tensorflow:global_step/sec: 463.392
INFO:tensorflow:loss = 0.00776011, step = 7701 (0.216 sec)
INFO:tensorflow:global_step/sec: 453.721
INFO:tensorflow:loss = 0.0035708607, step = 7801 (0.220 sec)
INFO:tensorflow:global_step/sec: 453.72
INFO:tensorflow:loss = 0.008371001, step = 7901 (0.220 sec)
INFO:tensorflow:global_step/sec: 423.37
INFO:tensorflow:loss = 0.002655299, step = 8001 (0.237 sec)
INFO:tensorflow:global_step/sec: 468.603
INFO:tensorflow:loss = 0.006116642, step = 8101 (0.212 sec)
INFO:tensorflow:global_step/sec: 439.754
INFO:tensorflow:loss = 0.049798254, step = 8201 (0.2

INFO:tensorflow:loss = 0.0019167841, step = 15301 (0.232 sec)
INFO:tensorflow:global_step/sec: 459.137
INFO:tensorflow:loss = 0.0030661107, step = 15401 (0.218 sec)
INFO:tensorflow:global_step/sec: 463.392
INFO:tensorflow:loss = 0.00614255, step = 15501 (0.216 sec)
INFO:tensorflow:global_step/sec: 458.295
INFO:tensorflow:loss = 0.0036711728, step = 15601 (0.218 sec)
INFO:tensorflow:global_step/sec: 431.406
INFO:tensorflow:loss = 0.005754758, step = 15701 (0.232 sec)
INFO:tensorflow:global_step/sec: 453.721
INFO:tensorflow:loss = 0.00050870056, step = 15801 (0.220 sec)
INFO:tensorflow:global_step/sec: 453.72
INFO:tensorflow:loss = 0.0007381078, step = 15901 (0.220 sec)
INFO:tensorflow:global_step/sec: 472.59
INFO:tensorflow:loss = 0.0076187854, step = 16001 (0.212 sec)
INFO:tensorflow:global_step/sec: 435.919
INFO:tensorflow:loss = 0.002553412, step = 16101 (0.229 sec)
INFO:tensorflow:global_step/sec: 478.928
INFO:tensorflow:loss = 0.00015055256, step = 16201 (0.209 sec)
INFO:tensorflow

INFO:tensorflow:loss = 0.0020732218, step = 23301 (0.216 sec)
INFO:tensorflow:global_step/sec: 453.721
INFO:tensorflow:loss = 0.0005531911, step = 23401 (0.220 sec)
INFO:tensorflow:global_step/sec: 453.72
INFO:tensorflow:loss = 0.0010950953, step = 23501 (0.220 sec)
INFO:tensorflow:global_step/sec: 453.721
INFO:tensorflow:loss = 0.00055587635, step = 23601 (0.220 sec)
INFO:tensorflow:global_step/sec: 450.857
INFO:tensorflow:loss = 0.00012828782, step = 23701 (0.222 sec)
INFO:tensorflow:global_step/sec: 461.255
INFO:tensorflow:loss = 0.0009460428, step = 23801 (0.217 sec)
INFO:tensorflow:global_step/sec: 453.72
INFO:tensorflow:loss = 0.0015272191, step = 23901 (0.236 sec)
INFO:tensorflow:global_step/sec: 435.919
INFO:tensorflow:loss = 0.001418497, step = 24001 (0.214 sec)
INFO:tensorflow:global_step/sec: 461.255
INFO:tensorflow:loss = 0.0005748706, step = 24101 (0.217 sec)
INFO:tensorflow:global_step/sec: 430.293
INFO:tensorflow:loss = 0.0014794224, step = 24201 (0.232 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 446.429
INFO:tensorflow:loss = 0.0004708774, step = 31301 (0.222 sec)
INFO:tensorflow:global_step/sec: 463.392
INFO:tensorflow:loss = 0.0013191457, step = 31401 (0.216 sec)
INFO:tensorflow:global_step/sec: 384.32
INFO:tensorflow:loss = 0.00015350487, step = 31501 (0.261 sec)
INFO:tensorflow:global_step/sec: 425.532
INFO:tensorflow:loss = 0.0003706299, step = 31601 (0.236 sec)
INFO:tensorflow:global_step/sec: 439.754
INFO:tensorflow:loss = 0.000660977, step = 31701 (0.227 sec)
INFO:tensorflow:global_step/sec: 429.922
INFO:tensorflow:loss = 0.00014320777, step = 31801 (0.233 sec)
INFO:tensorflow:global_step/sec: 439.368
INFO:tensorflow:loss = 0.0006632308, step = 31901 (0.226 sec)
INFO:tensorflow:global_step/sec: 471.698
INFO:tensorflow:loss = 0.000121571924, step = 32001 (0.212 sec)
INFO:tensorflow:global_step/sec: 430.663
INFO:tensorflow:loss = 0.00043609095, step = 32101 (0.233 sec)
INFO:tensorflow:global_step/sec: 460.829
INFO:tensorflow:loss = 0.0011

INFO:tensorflow:loss = 0.0003369895, step = 39201 (0.227 sec)
INFO:tensorflow:global_step/sec: 441.696
INFO:tensorflow:loss = 0.0004159444, step = 39301 (0.227 sec)
INFO:tensorflow:global_step/sec: 440.917
INFO:tensorflow:loss = 0.0004741313, step = 39401 (0.227 sec)
INFO:tensorflow:global_step/sec: 427.35
INFO:tensorflow:loss = 0.00016665139, step = 39501 (0.234 sec)
INFO:tensorflow:global_step/sec: 473.934
INFO:tensorflow:loss = 0.0006688069, step = 39601 (0.210 sec)
INFO:tensorflow:global_step/sec: 420.168
INFO:tensorflow:loss = 0.00013615772, step = 39701 (0.238 sec)
INFO:tensorflow:global_step/sec: 428.449
INFO:tensorflow:loss = 0.0011400488, step = 39801 (0.234 sec)
INFO:tensorflow:global_step/sec: 431.035
INFO:tensorflow:loss = 0.00075661315, step = 39901 (0.233 sec)
INFO:tensorflow:Saving checkpoints for 40000 into C:\Users\student\AppData\Local\Temp\tmp8cq4fsj7\model.ckpt.
INFO:tensorflow:Loss for final step: 0.00036211338.


SKCompat()

## 10.3 텐서플로의 저수준 API로 심층 신경망 훈련하기

직접 만들어보자

### 10.3.1 구성단계

In [9]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [10]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name="y")

- 플레이스홀더 X는 입력층의 역활을 하게 됨(실행 단계 동안 한 번에 하나씩 훈련 배치로 바뀌게 됨) 


In [11]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs) # 왜 표준편차를 이렇게 잡는지 모르겠음
        init = tf.truncated_normal((n_inputs, n_neurons), stddev= stddev)
        W = tf.Variable(init, name='kernel')
        b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return(Z)
        
## 저런 표준편차를 사용하면 알고리즘이 훨씬 빠르게 수렴함.
## 자세한 것은 11단원에서..

## 각 뉴런에 대해서 선형독립성이 유지되어야 함(그래서 truncated_normal을 써야함)

## 마지막으로 tf.nn.relu와 같은 activation 매개변수가 지정되어 있으면 activation(Z))를
## 반환함, 그렇지 않으면 그냥 Z를 반환함 

- 하나의 뉴런 층을 만드는 함수가 생김

In [12]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                          activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name='hidden2',
                          activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name='outputs')

- logits는 소프트맥스 활성화 함수로 들어가기 직전의 신경망 출력이다.
- 최적화 작업 때문에 소프트맥스 계산은 나중에..

- (n-th, 28 * 28) => (28 * 28, 300) => (300, 10)

- 예상했겠지만(?!), 텐서플로에는 표준 신경망 층을 만드는 편리한 함수가 있다.  
- tf.layers.dense() 함수는 모든 입력이 은닉층에 있는 모든 뉴런과 연결된 완전 연결 층을 만든다.
- 이 함수는 적절한 초기화 방식을 사용해 kernel과 bias라는 이름으로 가중치와 편향 변수를 만든다. 
- activation 매개변수로 활성화 함수를 지정할 수 있다.
- 11장에서 보겠지만 규제와 정규화 매개변수도 지원한다.

In [13]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1',
                             activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2',
                             activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

- 신경망 모델이 준비되었으니 훈련에 사용할 비용 함수를 정의하자.
- 4장의 소프트맥스 회귀에서처럼 크로스 엔트로피를 사용하자.
- 크로스 엔트로피는 모델이 타킷 클래스에 대해 낮은 확률을 추정하지 않도록 제약을 가한다.


- 텐서플로는 크로스 엔트로피를 계산하기 위해 여러 개의 함수를 제공한다. 여기서는 sparse_softmax_cross_entropy_logits()함수를 사용한다.
- 이 함수는 logit을 기반으로 크로스 엔트로피를 계산한다.
- 그리고 0에서 '클래스 수 -1' 사이의 정수로 된 레이블을 기대한다.
- 이 함수는 각 샘플에 대한 크로스 엔트로피를 담은 1D 텐서를 반환한다. 그런 다음 텐서플로우의 reduce_mean() 함수를 사용하여 모든 샘플에 대한 크로스 엔트로피 평균을 계산한다

In [14]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                             logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

In [15]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

- 마지막은 모델을 평가하는 방법을 지정하는 것이다.
- 여기서는 정확도를 사용해 성능을 측정한다. 

- 먼저 샘플마다 가장 큰 로짓이 타깃 클래스에 해당하는지 여부를 확인해 신경망의 예측이 맞는지 결정한다.
- 이를 위해 in_top_k() 함수를 사용한다.
- 이 함수는 불린 값으로 채워진 1D 텐서를 반환하므로 실수형으로 변환하고 평균을 낸다. 이 값이 신경망의 전체 정확도이다.

In [16]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1) ##logits의 shape는 (None, 10)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

- 통상적으로 모든 변수를 초기화하는 노드를 만들고 훈련된 모델 파라미터를 디스크에 저장하기 위한 Saver 객체를 생성한다.

In [17]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

- 정리하자면, 입력과 출력을 위한 플레이스홀더를 만들었고 뉴런의 층을 구성하는 함수를 만들고 이 함수를 사용해 심층 신경망을 생성했다. 그리고 비용 함수와 옵티마이저를 생성하고 마지막에 성능 지표를 정의했다.

### 10.3.2 실행 단계

In [18]:
n_epochs = 40
batch_size = 50

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size ## 배치 개수
    for batch_idx in np.array_split(rnd_idx, n_batches): ## rnd_idx 배열을 배치 개수만큼 자름
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [20]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_batch = accuracy.eval(feed_dict={X:X_batch, y:y_batch}) ## 이게 왜 가능한거지... ㅎㄷㄷ
        acc_valid = accuracy.eval(feed_dict={X:X_valid, y:y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_valid)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.902
1 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9218
2 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9314
3 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.9382
4 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.944
5 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9484
6 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9524
7 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9576
8 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9604
9 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9628
10 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.9638
11 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9654
12 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.965
13 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9698
14 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9696
15 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9722
16 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.973
17 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9722
18 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9726
19 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9728
20 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9746
21 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9742
22 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9748
23 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9756
24 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9744
25 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9762
26 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.975

In [21]:
for i in range(10):
    print(i)
print(i)
print(i)

0
1
2
3
4
5
6
7
8
9
9
9


### 10.3.3 신경망 사용하기

신경망을 훈련시키고 나면 이를 사용해 예측을 만들 수 있다. 이때 구성 단계는 그대로 재사용할 수 있지만, 실행 단계는 다음과 같이 바꿔야 한다.

In [24]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_new_scaled = X_test[:20] ## 새로운 이미지라면 0에서 1로 스케일 조정해야함
    Z = logits.eval(feed_dict={X:X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


모든 클래스에 대한 추정 확률을 알고 싶다면 로짓에 sotmax() 함수를 적용하면 된다. 하지만 어떤 클래스 하나를 예측하는 것이라면 간단하게 로짓 값이 가장 큰 클래스를 선택하면 된다.

In [23]:
print("예측 클래스:", y_pred)
print("진짜 클래스:", y_test[:20])

예측 클래스: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
진짜 클래스: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


## 10.4 신경망 하이퍼파라미터 튜닝하기

- 신경망은 조절해야 할 하이퍼파라미터가 많아진다. 상상할 수 있는 어떤 네트워크 토폴로지도 사용할 수 있을 뿐만 아니라 간단한 다층 퍼셉트론조차도 층 수나 층마다의 뉴런 수, 각 층에서 사용하는 활성화 함수, 가중치 초기화 방식 등을 바꿀 수 있다.
- 이전 장에서 한 것처럼 적절한 하이퍼파라미터를 찾기 위해 교차 검증을 활용한 그리드 탐색을 할 수 있지만, 조정할 하이퍼파라미터가 많고 대규모 데이터셋에 신경망을 훈련할 때 오랜 시간이 걸리기 때문에 주어진 시간 안에 전체 하이퍼파라미터 공간 중 작은 부분만 탐색할 수 있다.  
- 이런 경우엔 2장에서 이야기한 랜덤 탐색을 사용하는 것이 낫다.
- 아니면 __오스카__ 같은 도구를 사용할 수 있다.

### 10.4.1 은닉층의 수

- 많은 문제가 은닉층 하나로 시작해도 쓸 만한 결과를 얻을 수 있다. 사실 은닉층이 하나인 다층 퍼셉트론이라도 뉴런 수가 충분하면 아주 복잡한 함수도 모델링할 수 있다는 것이 밝혀졌다. 하지만 심층 신경망이 얕은 신경망보다 파라미터 효율성이 훨씬 좋다. 심층 신경망은 복잡한 함수를 모델링하는데 얕은 신경망보다 훨씬 적은 수의 뉴런을 사용하기 때문에 더 빠르게 훈련된다.

- 계층 구조는 심층 신경망이 좋은 솔루션으로 빨리 수렴하게 도와줄 뿐만 아니라 새로운 데이터에 일반화되는 능력도 향상시켜준다. 새로운 신경망의 처음 몇 개 층의 가중치와 편향을 난수로 초기화하는 대신 첫 번째 신경망의 층에 있는 가중치와 편향 값으로 초기화할 수 있다. 이런 방식을 사용하면 대부분의 사진에 나타나는 저수준 구조를 학습할 필요가 없다.
- 아주 복잡한 작업일 경우에는 비슷한 작업에서 가장 뛰어난 성능을 내는 미리 훈련된 네트워크 일부를 재사용하는 것이 일반적이다(11장에서 자세히 설명).

### 10.4.2 은닉층의 뉴런 수

- 은닉층의 구성 방식은 일반적으로 각 층의 뉴런을 점점 줄여서 깔때기처럼 구성한다. 저수준의 많은 특성이 고수준의 적은 특성으로 합쳐질 수 있기 때문이다.
- 하지만 이 구성은 요즘엔 일반적이지 않고, 모든 은닉층에 같은 크기를 사용한다.
- 하이퍼파라미터가 층마다 따로 있지 않아서 전체를 통틀어 하나만 조정하면 된다.
- 층의 개수와 마찬가지로 네트워크가 과대적합이 시작되기 전까지 점진적으로 뉴런 수를 늘린다.


- 일반적으로 층의 뉴런 수보다 층 수를 늘리는 쪽이 이득이 많다.


- 단순 접근 방식은 실제 필요한 것보다 더 많은 층과 뉴런을 가진 모델을 선택하고, 그런 다음 과대적합되지 않도록 조기 종료 기법을 사용하는 것이다. 이를 '스트레치 팬치'방식이라고 부른다(그리고 11장에서 볼 다른 규제, 특히 드롭아웃을 사용한다).

### 10.4.3 활성화 함수

- 대부분의 경우 은닉층에 ReLU 활성화 함수를 사용한다(또한 11장에 나오는 ReLU의 변종을 사용할 수 있다).
- 이 함수는 다른 활성화 함수보다 계산이 조금 더 빠르고, 입력값이 클 때 특정 값에 수렴하지 않는 덕분에 경사 하강법이 평편한 지역에서 심하게 지체되지 않는다.


- 출력층에서는 소프트맥스 활성화 함수가 일반적으로 분류 작업의 좋은 선택이다(클래스가 상호 배타적일 경우). 회귀 작업일 경우에는 활성화 함수를 사용하지 않는다.